In [2]:
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *
import os
import glob

In [3]:
conn = psycopg2.connect(database="postgres", user="bro", password="n9l1b8r1p", host="127.0.0.1", port="5432")

In [4]:
cur = conn.cursor()

In [5]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files:
            all_files.append(os.path.abspath(f))
    return all_files

In [14]:
song_files = get_files('data/song_data')

In [15]:
filepath = song_files[0]
print(filepath)

/home/bro/Documents/data_science_projects/data_engineering/data_modelling/data/song_data/A/A/C/TRAACTB12903CAAF15.json


In [16]:
from pyspark.sql import SparkSession
spark1 = SparkSession.builder.appName('etl').getOrCreate()

In [17]:
df =  spark1.read.json(filepath)

In [18]:
df.columns

['artist_id',
 'artist_latitude',
 'artist_location',
 'artist_longitude',
 'artist_name',
 'duration',
 'num_songs',
 'song_id',
 'title',
 'year']

In [29]:
df.show()

+------------------+---------------+---------------+----------------+----------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|artist_location|artist_longitude|     artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+---------------+----------------+----------------+---------+---------+------------------+--------------------+----+
|AR0RCMP1187FB3F427|       30.08615|   Beaumont, TX|       -94.10158|Billie Jo Spears|133.32853|        1|SOGXHEG12AB018653E|It Makes No Diffe...|1992|
+------------------+---------------+---------------+----------------+----------------+---------+---------+------------------+--------------------+----+



In [39]:
df.head()

Row(artist_id='AR0RCMP1187FB3F427', artist_latitude=30.08615, artist_location='Beaumont, TX', artist_longitude=-94.10158, artist_name='Billie Jo Spears', duration=133.32853, num_songs=1, song_id='SOGXHEG12AB018653E', title='It Makes No Difference Now', year=1992)

In [24]:
df.createOrReplaceTempView('song_data')

In [25]:
df_song = spark1.sql('SELECT song_id,title,artist_id,year,duration from song_data')

In [72]:
df_song.show()

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOGXHEG12AB018653E|It Makes No Diffe...|AR0RCMP1187FB3F427|1992|133.32853|
+------------------+--------------------+------------------+----+---------+



In [63]:
song_data = df_artist.head().asDict()

In [64]:
type(song_data)

dict

In [65]:
song_data

{'song_id': 'SOGXHEG12AB018653E',
 'title': 'It Makes No Difference Now',
 'artist_id': 'AR0RCMP1187FB3F427',
 'year': 1992,
 'duration': 133.32853}

In [66]:
[*song_data.values()]

['SOGXHEG12AB018653E',
 'It Makes No Difference Now',
 'AR0RCMP1187FB3F427',
 1992,
 133.32853]

In [102]:
filepath = song_files[0]
print(filepath)

/home/bro/Documents/data_science_projects/data_engineering/data_modelling/data/song_data/A/A/C/TRAACTB12903CAAF15.json


In [27]:
df_artist = spark1.sql('SELECT artist_id,artist_name,artist_location,artist_latitude,artist_longitude from song_data')

In [75]:
df_artist.show()

+------------------+----------------+---------------+---------------+----------------+
|         artist_id|     artist_name|artist_location|artist_latitude|artist_longitude|
+------------------+----------------+---------------+---------------+----------------+
|AR0RCMP1187FB3F427|Billie Jo Spears|   Beaumont, TX|       30.08615|       -94.10158|
+------------------+----------------+---------------+---------------+----------------+



In [76]:
artist_data = df_artist.head().asDict()

In [77]:
artist_data

{'artist_id': 'AR0RCMP1187FB3F427',
 'artist_name': 'Billie Jo Spears',
 'artist_location': 'Beaumont, TX',
 'artist_latitude': 30.08615,
 'artist_longitude': -94.10158}

In [78]:
[*artist_data.values()]

['AR0RCMP1187FB3F427', 'Billie Jo Spears', 'Beaumont, TX', 30.08615, -94.10158]

In [7]:
log_files = get_files('data/log_data')

In [8]:
filepath = log_files[0]

In [9]:
print(filepath)

/home/bro/Documents/data_science_projects/data_engineering/data_modelling/data/log_data/2018/11/2018-11-04-events.json


In [10]:
df = spark1.read.json(filepath)

In [11]:
df.columns

['artist',
 'auth',
 'firstName',
 'gender',
 'itemInSession',
 'lastName',
 'length',
 'level',
 'location',
 'method',
 'page',
 'registration',
 'sessionId',
 'song',
 'status',
 'ts',
 'userAgent',
 'userId']

In [109]:
df.show()

+--------------------+----------+---------+------+-------------+--------+----------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|      auth|firstName|gender|itemInSession|lastName|    length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+----------+---------+------+-------------+--------+----------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|                null| Logged In| Theodore|     M|            0|   Smith|      null| free|Houston-The Woodl...|   GET|    Home|1.540306145796E12|      154|                null|   200|1541290555796|Mozilla/5.0 (Wind...|    52|
|  Professor Longhair| Logged In|      Ann|     F|            0|   Banks| 214.20363| free|  Salt

In [12]:
df.head()

Row(artist=None, auth='Logged In', firstName='Theodore', gender='M', itemInSession=0, lastName='Smith', length=None, level='free', location='Houston-The Woodlands-Sugar Land, TX', method='GET', page='Home', registration=1540306145796.0, sessionId=154, song=None, status=200, ts=1541290555796, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='52')

In [111]:
df.createOrReplaceTempView('log_data')

In [1]:
df_user = spark1.sql('SELECT userId,firstName,lastName,gender,level FROM log_data')

NameError: name 'spark1' is not defined

In [113]:
df_user.show()

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    52| Theodore|   Smith|     M| free|
|    99|      Ann|   Banks|     F| free|
|    43|   Jahiem|   Miles|     M| free|
|    43|   Jahiem|   Miles|     M| free|
|    43|   Jahiem|   Miles|     M| free|
|    43|   Jahiem|   Miles|     M| free|
|     6|  Cecilia|   Owens|     F| free|
|     6|  Cecilia|   Owens|     F| free|
|    25|   Jayden|  Graves|     M| paid|
|    25|   Jayden|  Graves|     M| paid|
|    25|   Jayden|  Graves|     M| paid|
|      |     null|    null|  null| paid|
|      |     null|    null|  null| paid|
|    25|   Jayden|  Graves|     M| paid|
|    25|   Jayden|  Graves|     M| paid|
|    25|   Jayden|  Graves|     M| paid|
|    25|   Jayden|  Graves|     M| paid|
|    25|   Jayden|  Graves|     M| paid|
|    25|   Jayden|  Graves|     M| paid|
|    25|   Jayden|  Graves|     M| paid|
+------+---------+--------+------+-----+
only showing top

In [114]:
song_data_dict = [row.asDict() for row in df_user.collect()]

In [115]:
song_data_dict

[{'userId': '52',
  'firstName': 'Theodore',
  'lastName': 'Smith',
  'gender': 'M',
  'level': 'free'},
 {'userId': '99',
  'firstName': 'Ann',
  'lastName': 'Banks',
  'gender': 'F',
  'level': 'free'},
 {'userId': '43',
  'firstName': 'Jahiem',
  'lastName': 'Miles',
  'gender': 'M',
  'level': 'free'},
 {'userId': '43',
  'firstName': 'Jahiem',
  'lastName': 'Miles',
  'gender': 'M',
  'level': 'free'},
 {'userId': '43',
  'firstName': 'Jahiem',
  'lastName': 'Miles',
  'gender': 'M',
  'level': 'free'},
 {'userId': '43',
  'firstName': 'Jahiem',
  'lastName': 'Miles',
  'gender': 'M',
  'level': 'free'},
 {'userId': '6',
  'firstName': 'Cecilia',
  'lastName': 'Owens',
  'gender': 'F',
  'level': 'free'},
 {'userId': '6',
  'firstName': 'Cecilia',
  'lastName': 'Owens',
  'gender': 'F',
  'level': 'free'},
 {'userId': '25',
  'firstName': 'Jayden',
  'lastName': 'Graves',
  'gender': 'M',
  'level': 'paid'},
 {'userId': '25',
  'firstName': 'Jayden',
  'lastName': 'Graves',
  'gen

In [116]:
for records in song_data_dict:
    print([*records.values()])

['52', 'Theodore', 'Smith', 'M', 'free']
['99', 'Ann', 'Banks', 'F', 'free']
['43', 'Jahiem', 'Miles', 'M', 'free']
['43', 'Jahiem', 'Miles', 'M', 'free']
['43', 'Jahiem', 'Miles', 'M', 'free']
['43', 'Jahiem', 'Miles', 'M', 'free']
['6', 'Cecilia', 'Owens', 'F', 'free']
['6', 'Cecilia', 'Owens', 'F', 'free']
['25', 'Jayden', 'Graves', 'M', 'paid']
['25', 'Jayden', 'Graves', 'M', 'paid']
['25', 'Jayden', 'Graves', 'M', 'paid']
['', None, None, None, 'paid']
['', None, None, None, 'paid']
['25', 'Jayden', 'Graves', 'M', 'paid']
['25', 'Jayden', 'Graves', 'M', 'paid']
['25', 'Jayden', 'Graves', 'M', 'paid']
['25', 'Jayden', 'Graves', 'M', 'paid']
['25', 'Jayden', 'Graves', 'M', 'paid']
['25', 'Jayden', 'Graves', 'M', 'paid']
['25', 'Jayden', 'Graves', 'M', 'paid']
['25', 'Jayden', 'Graves', 'M', 'paid']
['25', 'Jayden', 'Graves', 'M', 'paid']
['25', 'Jayden', 'Graves', 'M', 'paid']
['25', 'Jayden', 'Graves', 'M', 'paid']
['25', 'Jayden', 'Graves', 'M', 'paid']
['25', 'Jayden', 'Graves', 

In [117]:
df.show()

+--------------------+----------+---------+------+-------------+--------+----------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|      auth|firstName|gender|itemInSession|lastName|    length|level|            location|method|    page|     registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+----------+---------+------+-------------+--------+----------+-----+--------------------+------+--------+-----------------+---------+--------------------+------+-------------+--------------------+------+
|                null| Logged In| Theodore|     M|            0|   Smith|      null| free|Houston-The Woodl...|   GET|    Home|1.540306145796E12|      154|                null|   200|1541290555796|Mozilla/5.0 (Wind...|    52|
|  Professor Longhair| Logged In|      Ann|     F|            0|   Banks| 214.20363| free|  Salt

In [118]:
df_ts = spark1.sql('SELECT ts FROM log_data where page=="NextSong"')

In [119]:
df_ts.show()

+-------------+
|           ts|
+-------------+
|1541292603796|
|1541300092796|
|1541300337796|
|1541300540796|
|1541306152796|
|1541310546796|
|1541310708796|
|1541310957796|
|1541311164796|
|1541311441796|
|1541311623796|
|1541311812796|
|1541312045796|
|1541312311796|
|1541312592796|
|1541312817796|
|1541312981796|
|1541313231796|
|1541313299796|
|1541313517796|
+-------------+
only showing top 20 rows



In [124]:
song_data_dict = [row.asDict() for row in df_ts.collect()]

In [125]:
song_data_dict

[{'ts': 1541292603796},
 {'ts': 1541300092796},
 {'ts': 1541300337796},
 {'ts': 1541300540796},
 {'ts': 1541306152796},
 {'ts': 1541310546796},
 {'ts': 1541310708796},
 {'ts': 1541310957796},
 {'ts': 1541311164796},
 {'ts': 1541311441796},
 {'ts': 1541311623796},
 {'ts': 1541311812796},
 {'ts': 1541312045796},
 {'ts': 1541312311796},
 {'ts': 1541312592796},
 {'ts': 1541312817796},
 {'ts': 1541312981796},
 {'ts': 1541313231796},
 {'ts': 1541313299796},
 {'ts': 1541313517796},
 {'ts': 1541313538796},
 {'ts': 1541313667796},
 {'ts': 1541313798796},
 {'ts': 1541314050796},
 {'ts': 1541314272796},
 {'ts': 1541314485796},
 {'ts': 1541314808796},
 {'ts': 1541315044796},
 {'ts': 1541315293796},
 {'ts': 1541315503796},
 {'ts': 1541315691796},
 {'ts': 1541315904796},
 {'ts': 1541316143796},
 {'ts': 1541316443796},
 {'ts': 1541316691796},
 {'ts': 1541316941796},
 {'ts': 1541317207796},
 {'ts': 1541317524796},
 {'ts': 1541323143796},
 {'ts': 1541323401796},
 {'ts': 1541323569796},
 {'ts': 15413236

In [126]:
type(song_data_dict[0])

dict

In [127]:
import datetime

In [136]:
for records in song_data_dict:
    date_list = []
    date = datetime.datetime.fromtimestamp([*records.values()][0]/1000.0)
    date_list = [date.strftime("%c"),date.strftime("%H"),date.strftime("%d"),date.strftime("%W"),date.strftime("%-m"),date.strftime("%Y"),date.strftime("%w")]
    #date_list = [date.strftime(),date.hour,date.day,date.isocalendar()[1],date.month,date.year,date.weekday()]
    #hour = date.weekday()
    #date = date.strftime('%Y-%m-%d %H:%M:%S')
    #cur.execute(time_table_insert, artist_data_list)
    print(date_list)

['Sun Nov  4 01:50:03 2018', '01', '04', '44', '11', '2018', '0']
['Sun Nov  4 03:54:52 2018', '03', '04', '44', '11', '2018', '0']
['Sun Nov  4 03:58:57 2018', '03', '04', '44', '11', '2018', '0']
['Sun Nov  4 04:02:20 2018', '04', '04', '44', '11', '2018', '0']
['Sun Nov  4 05:35:52 2018', '05', '04', '44', '11', '2018', '0']
['Sun Nov  4 06:49:06 2018', '06', '04', '44', '11', '2018', '0']
['Sun Nov  4 06:51:48 2018', '06', '04', '44', '11', '2018', '0']
['Sun Nov  4 06:55:57 2018', '06', '04', '44', '11', '2018', '0']
['Sun Nov  4 06:59:24 2018', '06', '04', '44', '11', '2018', '0']
['Sun Nov  4 07:04:01 2018', '07', '04', '44', '11', '2018', '0']
['Sun Nov  4 07:07:03 2018', '07', '04', '44', '11', '2018', '0']
['Sun Nov  4 07:10:12 2018', '07', '04', '44', '11', '2018', '0']
['Sun Nov  4 07:14:05 2018', '07', '04', '44', '11', '2018', '0']
['Sun Nov  4 07:18:31 2018', '07', '04', '44', '11', '2018', '0']
['Sun Nov  4 07:23:12 2018', '07', '04', '44', '11', '2018', '0']
['Sun Nov 

In [19]:
df.head()

Row(artist_id='AR0RCMP1187FB3F427', artist_latitude=30.08615, artist_location='Beaumont, TX', artist_longitude=-94.10158, artist_name='Billie Jo Spears', duration=133.32853, num_songs=1, song_id='SOGXHEG12AB018653E', title='It Makes No Difference Now', year=1992)

In [20]:
df.columns

['artist_id',
 'artist_latitude',
 'artist_location',
 'artist_longitude',
 'artist_name',
 'duration',
 'num_songs',
 'song_id',
 'title',
 'year']

In [21]:
df.show()

+------------------+---------------+---------------+----------------+----------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|artist_location|artist_longitude|     artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+---------------+----------------+----------------+---------+---------+------------------+--------------------+----+
|AR0RCMP1187FB3F427|       30.08615|   Beaumont, TX|       -94.10158|Billie Jo Spears|133.32853|        1|SOGXHEG12AB018653E|It Makes No Diffe...|1992|
+------------------+---------------+---------------+----------------+----------------+---------+---------+------------------+--------------------+----+



In [28]:
df_song.columns

['song_id', 'title', 'artist_id', 'year', 'duration']

In [29]:
df_artist.columns

['artist_id',
 'artist_name',
 'artist_location',
 'artist_latitude',
 'artist_longitude']

In [30]:
df.columns

['artist_id',
 'artist_latitude',
 'artist_location',
 'artist_longitude',
 'artist_name',
 'duration',
 'num_songs',
 'song_id',
 'title',
 'year']

In [32]:
for row in df.rdd.collect():
    print(row)

Row(artist_id='AR0RCMP1187FB3F427', artist_latitude=30.08615, artist_location='Beaumont, TX', artist_longitude=-94.10158, artist_name='Billie Jo Spears', duration=133.32853, num_songs=1, song_id='SOGXHEG12AB018653E', title='It Makes No Difference Now', year=1992)
